In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "round summary" in dict.keys():
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 5
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You invited Professor Dagger into the storage room, the space still cluttered despite your earlier attempt to clear some of the items out. The two of you began scouring through old furniture, leaving nothing unturned, yet, after two long hours, your hands held nothing but dust. You sensed the Professor’s eyes laced with disappointment, perhaps he has already given up the search, for to encounter a rare artifact like the one he described must require great deal of luck. But your desire for a better life fuels your persistence to hunt the fourth book down. You revisit the drawers you may have overlooked, potential compartments that are hidden behind intricate mechanisms like the seance table. And as you attempt to wrestle a large wooden shelf to another wall, your slip of the hand causes it to collide with the wall behind it, producing a hollow thud. Professor Dagger immediately placed his hands against the false wall, sliding across its surface gently like one would a body. Soon he sens

I select option 1. TURN: 1 


You decide to follow Professor Dagger closely in silence, your steps cautious and measured as you navigate the dimly lit corridor. The air is thick with a musty, decayed scent that seems to cling to the back of your throat, making each breath a conscious effort. The walls are lined with peculiar markings that seem to dance in the shadows, their origins and meanings obscured by the dim light. As you proceed, the corridor narrows, forcing you and the Professor to move in a single file. 

The passage eventually opens into a small, hidden room. The sight before you is ghastly; at the center of the room lies the withered body of what you assume to be the storage room's previously deceased owner's sister. Her face is completely gone, replaced by a gaping cavity from which something sinister writhes within. Below her, a pentagram is drawn in a strange red fluid, its purpose unfathomable but undoubtedly macabre. At the far end of the room, you spot the fourth book

I am making a skill check using SANITY against a difficulty of HARD.
And I rolled a 27 for a result of FAILURE. TURN: 2 


As you fail to maintain your composure upon witnessing the grotesque scene, your mind reels from the horrific sight. The writhing entity within the cavity of the withered body's face, coupled with the sinister pentagram, sends you into a spiral of terror and disbelief. Your heart pounds against your chest, your breaths become shallow and rapid, and for a moment, the world seems to tilt on its axis. The very fabric of reality appears to warp around you, as the horrors of what lay before you seep into your psyche, leaving an indelible mark on your sanity.

The Professor, noticing your distress, places a reassuring hand on your shoulder, attempting to ground you back to reality. However, his touch barely registers over the maelstrom of panic and dread that consumes you.

Despite the overwhelming fear, you realize you must act, but the question remains: what will you d

I select option 1. TURN: 3 


skill: sanity
difficulty: hard

I am making a skill check using SANITY against a difficulty of HARD.
And I rolled a 22 for a result of SUCCESS. TURN: 4 


With monumental effort, you manage to pull yourself together, pushing past the fear that threatens to overwhelm you. The horrors that lay before you, though not forgotten, are momentarily pushed to the back of your mind as you focus on the task at hand. Your training and experiences as an ex-marine kick in, allowing you to compartmentalize the terror and approach the situation with a semblance of clarity and determination.

You step forward, your eyes fixed on the podium at the far end of the room, where the fourth book rests. Each step feels heavier than the last, not just from the gravity of the situation, but from the weight of your own resolve. As you reach the podium, you take a moment to examine the book. Its cover is adorned with symbols that seem to pulsate with an eerie, otherworldly energy, yet you find yourself drawn to it, compelled by the possibility o

I select option 3. TURN: 5 


As you reach out and take the book from the podium, a palpable tension fills the air. The moment your fingers brush against the ancient tome, the strange red fluid forming the pentagram beneath the withered body begins to evaporate, disappearing without a trace as if it were never there. The room's atmosphere shifts, a sense of foreboding enveloping you as the sinister presence within the cavity of the withered body's face stirs.

Without warning, tentacles burst forth from the gaping maw where the face once was, writhing and flailing with malevolent intent. Professor Dagger, who had been observing from a safe distance, has no time to react as the tentacles ensnare him, pulling him towards the horrific visage with terrifying speed. His screams fill the room, a chilling reminder of the danger you both faced in this hidden chamber.

In a matter of seconds, the Professor is consumed, leaving you alone with the now animate and malevolent entity. The horror of 

run towards the exit. TURN: 6 


You don't waste a moment. With the book clutched tightly in your hand, you turn on your heels and sprint towards the corridor, the echoes of your footsteps bouncing off the narrow walls. The entity lets out an otherworldly screech behind you, its tentacles slithering across the ground with alarming speed in pursuit. The air grows colder as you run, the darkness of the corridor seemingly stretching on forever, a never-ending escape from the nightmare that chases you.

Your heart pounds in your chest, a frenzied rhythm that matches your desperate pace. You dare not look back, for fear that seeing the entity might paralyze you with terror. Instead, you focus on the faint light at the end of the corridor, the promise of escape, of survival. 

As you burst out of the corridor and into the main storage room, you realize you must find a way to lock the entity behind or somehow barricade the entrance to the hidden room.

What will you do next?

possible actions

I select option 2. TURN: 7 


With the entity's otherworldly screeches echoing behind you, you make a split-second decision to head straight for the storage facility's exit. The weight of the book in your hand serves as a constant reminder of the perilous journey you've undertaken, and now, the urgency to escape with your life.

Your lungs burn with each breath, a testament to the sheer willpower driving you forward. The storage room, once a place of hope and potential fortune, now feels like a labyrinthine trap, its cluttered aisles and dim lighting creating an oppressive atmosphere. But your focus remains unwavering, the exit your sole beacon of hope.

Finally, you reach the heavy, metal door that serves as the only barrier between you and the outside world. With a surge of adrenaline, you push against it, only to find...

The door is locked. Panic sets in as the realization hits you; there is no other way out. The entity's relentless pursuit leaves you with precious little time to t

I select option 2. TURN: 8 


In a desperate attempt to escape, you look around frantically for anything that could be used to break the door open. Your eyes land on a heavy, rusted pipe lying amongst the debris, a remnant of the storage room's forgotten utilities. With no time to spare, you grab it, feeling its weight grounding you in the reality of your dire situation.

With the entity's shrieks growing louder, signaling its imminent approach, you position yourself before the locked door. You raise the pipe with both hands, channeling every ounce of strength and desperation into your muscles. The first strike against the door sends a jarring vibration up your arms, but the door holds firm. You strike again, and then again, each hit a cacophony of metal against metal, a desperate symphony for survival.

On the fourth swing, the lock mechanism begins to give way, its integrity compromised by the relentless assault. With a final, Herculean effort, you swing the pipe once more, and this 